In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import mean_squared_error
from scipy import stats
import os

In [ ]:
# file = pd.read_excel("Files/Data/ComputoGobernador2015_Casilla (1).xlsx", sheet_name = "POR CASILLA", skiprows=1,skipfooter=1) # Alexis
file = pd.read_csv("Files/Data/ComputoGobernador2015_Casilla (1).csv")
muestra_actual = pd.read_csv("Files/Exports/PAQUETES.csv")

In [ ]:
muestra_actual

In [ ]:
df_dic = muestra_actual.groupby("DISTRITO.LOCAL").agg({"PAN":"count"}).reset_index().rename(columns={"PAN" : "TOTAL"})
df_dic

In [ ]:
sim = pd.DataFrame()
# Declara el DataFrame de varianzas
varianzas = pd.DataFrame()
# Crear dataframe muestra
muestra = pd.DataFrame()
# Crear dataframe muestra distritos
muestra_distritos = pd.DataFrame()
for index,row in df_dic.iterrows():
    total_distrito = file[file["DISTRITO LOCAL"] == row[0]].iloc[:,:]
    # obtener muestra por distrito
    muestra_dis = muestra_actual[muestra_actual["DISTRITO.LOCAL"] == row[0]]
    m1 = muestra_dis.iloc[:, 5:]
    cociente = len(total_distrito)/len(m1)
    suma_dis = cociente*(m1.sum())
    # agregamos a nuestros dataframes de la simulacion actual
    muestra_distritos = muestra_distritos.append(suma_dis, ignore_index = True)
    # agregamos la muestra con todos los datos
    muestra = muestra.append(muestra_dis, ignore_index = True)

# calculamos la proporcion de la simulacion
X_gorro = muestra_distritos.sum()["VOTACION.TOTAL.EMITIDA"]
# Una Y__gorro por partido
Y_gorros = muestra_distritos.sum()
suma = Y_gorros / X_gorro
sim = sim.append(suma, ignore_index = True)

# Calculamos la varianza
varianza_distritos = pd.DataFrame()
for index,row in df_dic.iterrows():
    total_distrito = file[file["DISTRITO LOCAL"] == row[0]].iloc[:,:]
    tmp_dis_varianza = muestra[muestra["DISTRITO.LOCAL"] == row[0]].iloc[:,5:]
    Nh = len(total_distrito)
    nh = row[1]

    tmp_var = pd.DataFrame()
    for index, row_var in tmp_dis_varianza.iterrows():
        G_hi = (row_var - (suma * row_var["VOTACION.TOTAL.EMITIDA"])) / X_gorro
        tmp_var = tmp_var.append(G_hi, ignore_index = True)

    # Calculamos las Gh
    G_h = tmp_var.sum()/nh
    # Data frame para guardar la resta al cuadrado de las (Ghi - Gh)^2
    tmp_var_square = pd.DataFrame()
    for index, row_var in tmp_var.iterrows():
        sub_square_tmp = (row_var - G_h) ** 2
        tmp_var_square = tmp_var_square.append(sub_square_tmp, ignore_index = True)

    var_Ghi = tmp_var_square.sum()/(row[1] - 1)
    var_distrito = (Nh ** 2) * ((1/nh) - (1/Nh)) * var_Ghi
    varianza_distritos = varianza_distritos.append(var_distrito, ignore_index = True)

varianzas = varianzas.append(varianza_distritos.sum(), ignore_index = True)

In [ ]:
sim

In [ ]:
varianzas

In [ ]:
delta = 2.575 * (varianzas ** (1/2))
delta

In [ ]:
# --- INTERVALOS ---
intervalo_PAN = pd.DataFrame({"estimacion": sim["PAN"][:].array, "delta": delta["PAN"].array})
intervalo_PAN["minimo"] = intervalo_PAN["estimacion"] - intervalo_PAN["delta"]
intervalo_PAN["maximo"] = intervalo_PAN["estimacion"] + intervalo_PAN["delta"]
intervalo_PAN["muestra"] = len(muestra_actual)
print(intervalo_PAN.muestra[0])

intervalo_COALICION = pd.DataFrame({"estimacion": sim["COALICION"][:].array, "delta": delta["COALICION"].array})
intervalo_COALICION["minimo"] = intervalo_COALICION["estimacion"] - intervalo_COALICION["delta"]
intervalo_COALICION["maximo"] = intervalo_COALICION["estimacion"] + intervalo_COALICION["delta"]
intervalo_COALICION["muestra"] = len(muestra_actual)
intervalo_COALICION.muestra[0]

In [ ]:
path = "Files/Exports/Res_PAN.csv"
historial_PAN = pd.DataFrame()
if os.path.exists(path):
    historial_PAN = pd.read_csv(path)
    if intervalo_PAN.muestra[0] not in historial_PAN.muestra.values:
        historial_PAN = historial_PAN.append(intervalo_PAN, ignore_index=True)
else :
    historial_PAN = historial_PAN.append(intervalo_PAN, ignore_index=True)

historial_PAN.to_csv(path, index=False)
historial_PAN

In [ ]:
path = "Files/Exports/Res_COALICION.csv"
historial_COALICION = pd.DataFrame()
if os.path.exists(path):
    historial_COALICION = pd.read_csv(path)
    if intervalo_COALICION.muestra[0] not in historial_COALICION.muestra.values:
        historial_COALICION = historial_COALICION.append(intervalo_COALICION, ignore_index=True)
else :
    historial_COALICION = historial_COALICION.append(intervalo_COALICION, ignore_index=True)
historial_COALICION.to_csv(path, index=False)
historial_COALICION

In [ ]:
dl = pd.read_excel("Files/Data/Casillas.xlsx")
del dl["Unnamed: 4"]
dl.columns = ["NOMBRE", "CODIGO", "CAPTURADAS", "MUESTRA"]
del dl["CAPTURADAS"]
dl

In [ ]:
contadas = muestra_actual
contadas["CASILLAS"] = 1
contadas = contadas[["DISTRITO.LOCAL", "CASILLAS"]]
contadas = contadas.groupby("DISTRITO.LOCAL").sum()
contadas.reset_index(inplace=True)
contadas.columns = ["NOMBRE","CASILLAS"]
contadas

In [ ]:
dl = dl.merge(contadas, on="NOMBRE")
dl

In [ ]:
dl["PORCENTAJE"] = (dl["CASILLAS"]/dl["MUESTRA"])*100
dl

In [ ]:
import fiona
fiona.supported_drivers['libkml'] = 'rw'
fiona.supported_drivers['LIBKML'] = 'rw'
fiona.supported_drivers['KML'] = 'rw'
gdl = gpd.read_file("Files/Poly/DISTRITOS_SONORA.kml")
gdl = gdl[["Name", "geometry"]]
gdl = gdl.sort_values("Name").reset_index(drop=True)
gdl["Porcentaje"] = 0.0

In [ ]:
for index, row in dl.iterrows():
    gdl.loc[gdl["Name"] == row["CODIGO"], "Porcentaje"] = row["PORCENTAJE"]

In [ ]:
gdl["center"] = gdl["geometry"].centroid
gdl_points = gdl.copy()
gdl_points.set_geometry("center", inplace = True)
gdl

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(20,12))

divider = make_axes_locatable(ax)

#cax = divider.append_axes("right", size="5%", pad=0.1)

base = gdl.plot(figsize=(20, 20),
         edgecolor = "black",
         column="Porcentaje",
         ax=ax,
         legend=True,
         cmap="Spectral",
         vmax=100,
         vmin=0,
         legend_kwds={'label': "Porcentaje",
                      'orientation': "vertical"})
# *************** ESTA PARTE ES PARA AGREGAR LAS LEYENDAS A CADA DISTRITO ***************************
texts = []
for x, y, label in zip(gdl_points.geometry.x, gdl_points.geometry.y, gdl_points["Name"]):
    texts.append(plt.text(x, y, label, fontsize = 8))
# ***************************************************************************************************
    
plt.axis("off")

In [ ]:
gdl

In [ ]:
color = plt.get_cmap("Spectral")
colors = [ color(i/100) for i in gdl.Porcentaje.values ]
y = dl["PORCENTAJE"]
figura = dl.plot.bar(figsize=(20,10),
            rot=0,
            x="CODIGO",
            y="PORCENTAJE",
            ylim=(0,100),
            color=colors)

In [ ]:
df_result = pd.DataFrame()
df_result = df_result.append(intervalo_PAN, ignore_index=True)
df_result = df_result.append(intervalo_COALICION, ignore_index=True)
df_result.index = ['PAN','COALICION']
df_result

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.errorbar(x=historial_PAN.muestra, y=historial_PAN["estimacion"],yerr=historial_PAN["delta"], label="PAN")
ax.errorbar(x=historial_COALICION.muestra, y=historial_COALICION["estimacion"],yerr=historial_COALICION["delta"], label="COALICION")
ax.legend(numpoints=1)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

rects1 = ax.bar( x=df_result.index, height=df_result.estimacion.values, color=['blue','orange'])

In [ ]:
import plotly.graph_objects as go

fig_plotly = go.Figure()

fig_plotly.add_trace(go.Scatter(
        name='PAN',
        x=historial_PAN.muestra,
        y=historial_PAN["estimacion"],
        error_y=dict(
            type='data',
            array=historial_PAN["delta"],
            visible=True
        )
    ))
fig_plotly.add_trace(go.Scatter(
        name='COALICION',
        x=historial_COALICION.muestra,
        y=historial_COALICION["estimacion"],
        error_y=dict(
            type='data',
            array=historial_COALICION["delta"],
            visible=True
        )
    ))

fig_plotly.show()